# **Homework 1 - task 25**
The main goal of this homework is to transform existing evaluation datasets into a format suitable for evaluating the linguistic skills of Large Language Models (LLMs) by reframing tasks as multi-choice Question Answering (QA) tasks, providing effective prompts, and generating distractors where necessary, all formatted in JSON Lines standard for submission.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Dependencies
import importlib
import subprocess

# List of dependencies
dependencies = ['torch', 'transformers', 'peft', 'accelerate', 'optimum', 'auto-gptq']

# Function to check and install dependencies
def check_and_install(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        subprocess.check_call(["pip", "install", package])

# Check and install dependencies
for dependency in dependencies:
    check_and_install(dependency)

# Additional installations for standard libraries
# Note: These are generally available with Python installation, but checking just in case.
standard_libraries = ['xml.etree.ElementTree', 'json']

for lib in standard_libraries:
    importlib.import_module(lib)


# **1. Data Loading:**


First we need to download the datasets from evalita to our python eviroment



In [3]:
!unzip task-25-textentail-20240322T151548Z-001.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


## 1.1 visualize initial data

**dev.xml** is the training (development) dataset.


**test_gold.xml** is the test dataset.

In [4]:
def vizualize_original_data(file_path):
  # Open the file and read its contents
  with open(file_path, 'r') as file:
      count = 0
      print_line = False

      for line in file:
          if '<pair' in line:
              print_line = True

              count += 1

          if '</pair>' in line:
              print(line.strip())
              print_line = False

          if print_line:
              print(line.strip())

          if count == 3:
              break

# Check info in dev.xml
file_path = 'task-25-textentail/dev.xml'
vizualize_original_data(file_path)

<pair entailment="YES" id="1001" task="WIKI">
<t>'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.</t>
<h>'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.</h>
</pair>
<pair entailment="YES" id="1002" task="WIKI">
<t>Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.</t>
<h>Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.</h>
</pair>
<pair entailment="YES" id="1003" task="WIKI">


In [5]:
# Check info in test_gold.xml
file_path = 'task-25-textentail/test_gold.xml'
vizualize_original_data(file_path)

<pair id="1401" task="WIKI" entailment="NO">
<t>Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.</t>
<h>Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.</h>
</pair>
<pair id="1402" task="WIKI" entailment="NO">
<t>Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.</t>
<h>Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.</h>
</pair>
<pair id="1403" task="WIKI" entailment="YES">


## 1.2 Data Reframing:
  First we need to change the type of the file from xml to json. Then we want to convert that json into the following format:
```JSON
{
    "id":           int,
    "text":         str,
    "hypothesis":   str,
    "choices":      list[str],
    "label":        int
}
```

In [6]:
import importlib

# Check if the package is installed, and install it if it is not
def check_and_install(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        import subprocess
        subprocess.check_call(["pip", "install", package])

check_and_install('xmltodict')


xmltodict is not installed. Installing...


In [7]:
import xml.etree.ElementTree as ET
import json

def xml_to_jsonl(xml_file, jsonl_file):
    with open(xml_file, 'r', encoding='utf-8') as f:
        tree = ET.parse(f)
        root = tree.getroot()

        with open(jsonl_file, 'w', encoding='utf-8') as jsonl_f:
            for pair in root.findall('pair'):
                json_data = {
                    'entailment': pair.get('entailment'),
                    'id': pair.get('id'),
                    't': pair.find('t').text.strip(),
                    'h': pair.find('h').text.strip()
                }
                jsonl_f.write(json.dumps(json_data) + '\n')

input_xml_path = 'task-25-textentail/dev.xml'
output_json_path = 'dev_out.jsonl'
xml_to_jsonl(input_xml_path, output_json_path)

input_xml_path = 'task-25-textentail/test_gold.xml'
output_json_path = 'test_gold_out.jsonl'
xml_to_jsonl(input_xml_path, output_json_path)


## 1.3 Visualize jsonl information

In [8]:
import json

def visualize_data(jsonl_file):
    with open(jsonl_file, 'r') as f:
        for line in f:
            data = json.loads(line)
            print("ID:", data["id"])
            print("Entailment:", data["entailment"])
            print("T:", data["t"])
            print("H:", data["h"])
            print()

jsonl_file = 'dev_out.jsonl'
visualize_data(jsonl_file)

ID: 1001
Entailment: YES
T: 'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.
H: 'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.

ID: 1002
Entailment: YES
T: Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.
H: Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.

ID: 1003
Entailment: YES
T: Viene eletto sindaco di Catanzaro al ballottaggio del giugno del 2006, con il 50,8% dei voti.
H: Viene eletto sindaco di Catanzaro al ballottaggio del giugno 2006, con il 50,8% dei voti.

ID: 1004
Entailment: YES
T: Alle elezioni politiche del 9 aprile e 10 aprile 2006 sarà candidato alla Camera nella lista de L'Ulivo in Toscana.
H: Alle elezioni politiche del 9 e 10 aprile 2006 sarà candidato alla Camera nella lista de L'Ulivo in Toscana.

ID: 1005
Entailment: YES
T: Nel gennaio del 1995 un regolare congresso del PSDI mise definitivamente in minoranza la corrente moderata

In [9]:
json_file = 'test_gold_out.jsonl'
visualize_data(json_file)

ID: 1401
Entailment: NO
T: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.
H: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.

ID: 1402
Entailment: NO
T: Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.
H: Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.

ID: 1403
Entailment: YES
T: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Como, nel 1967, riprese a giocare nelle fila del FC Chiasso nella Serie A Svizzera.
H: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Lecco, nel 1967, tornò a giocare nelle fila del FC Chiasso nella Serie A Svizzera.

ID: 1404
Entailment: NO
T: Giuseppe Cavanna è stato un calciatore, attivo negli anni '30, nel ruolo di portiere.
H: Giovanni Cavanna è stato un calciatore, attivo negli anni '30, nel ruolo di portiere.

ID: 1405
Entailment: NO
T: Attualmente ap


## 1.4 Format jsonl

In order to get the json file with the desired format:

1. Set id to be an identifier of each pair. (from 0 to the number of pairs)
2. Rename entailment to label and change YES or NO to 0 or 1 respectivly
3. Replace t with text.
4. Replace h with hypothesis.
5. Add a choices key with entailed or not entailed


In [10]:
import json

# Rearrange the JSONL file content and write to another JSONL file
def rearrange_jsonl(input_file_path, output_file_path):
    id_counter = 0
    with open(input_file_path, "r") as input_file:
        with open(output_file_path, "w") as output_file:
            for line in input_file:
                data = json.loads(line)
                new_data = {
                    #"id": int(data['id']),
                    "id": id_counter, # set id as an increment int
                    "text": data['t'],
                    "hypothesis": data['h'],
                    "choices": ["implicato", "non implicato"],
                    "label": 0 if data['entailment'] == "YES" else 1
                }
                id_counter += 1
                json.dump(new_data, output_file)
                output_file.write('\n')

input_jsonl_file = 'test_gold_out.jsonl'
output_jsonl_file = 'textual_entailment-task1-test-data.jsonl'
rearrange_jsonl(input_jsonl_file, output_jsonl_file)

input_jsonl_file = 'dev_out.jsonl'
output_jsonl_file = 'textual_entailment-task1-train-data.jsonl'
rearrange_jsonl(input_jsonl_file, output_jsonl_file)


## 1.5 Visualize final jsonl

In [11]:
import json

def visualize_data(jsonl_file):
    with open(jsonl_file, 'r') as f:
        line_count = 0
        for line in f:
            data = json.loads(line)
            print(data)
            line_count += 1
            if line_count == 5:
                break

jsonl_file = 'textual_entailment-task1-test-data.jsonl'
visualize_data(jsonl_file)

{'id': 0, 'text': "Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.", 'hypothesis': "Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.", 'choices': ['implicato', 'non implicato'], 'label': 1}
{'id': 1, 'text': 'Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.', 'hypothesis': 'Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.', 'choices': ['implicato', 'non implicato'], 'label': 1}
{'id': 2, 'text': "Bruschini però non si perse d'animo e, tornato sulle rive del lago di Como, nel 1967, riprese a giocare nelle fila del FC Chiasso nella Serie A Svizzera.", 'hypothesis': "Bruschini però non si perse d'animo e, tornato sulle rive del lago di Lecco, nel 1967, tornò a giocare nelle fila del FC Chiasso nella Serie A Svizzera.", 'choices': ['implicato', 'non implicato'], 'label': 0}
{'id': 3, 'text': "Giuseppe Cavanna è stato un calciatore, 

In [12]:
json_file = 'textual_entailment-task1-train-data.jsonl'
visualize_data(json_file)

{'id': 0, 'text': "'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.", 'hypothesis': "'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.", 'choices': ['implicato', 'non implicato'], 'label': 0}
{'id': 1, 'text': 'Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.', 'hypothesis': 'Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.', 'choices': ['implicato', 'non implicato'], 'label': 0}
{'id': 2, 'text': 'Viene eletto sindaco di Catanzaro al ballottaggio del giugno del 2006, con il 50,8% dei voti.', 'hypothesis': 'Viene eletto sindaco di Catanzaro al ballottaggio del giugno 2006, con il 50,8% dei voti.', 'choices': ['implicato', 'non implicato'], 'label': 0}
{'id': 3, 'text': "Alle elezioni politiche del 9 aprile e 10 aprile 2006 sarà candidato alla Camera nella lista de L'Ulivo in Toscana.", 'hypothesis': "Alle elezioni politiche del 9 e 10 aprile 2006 sarà cand

## **2. Promt formulation**

In order to use this dataset we need to generate three prompts that can be used to get if a text is entailed to a hypothesis and then insert them into a json file. The three prompts are:

**Template 1:**


Prompt: "La frase '{text}' sostiene la frase '{hypothesis}'?"

Translation: "Does the statement '{text}' entail the statement '{hypothesis}'?"4

**Template 2:**


Prompt: "La frase '{text}' implica la frase '{hypothesis}'?"

Translation: "Does the statement '{text}' imply the statement '{hypothesis}'?"

**Template 3:**


Prompt: "La frase '{hypothesis}' può essere dedotta dalla frase '{text}'?"

Translation: "Can the statement '{hypothesis}' be deduced from the statement '{text}'?"


In [13]:
# Function to generate a JSON Lines file with a list of prompts
def generate_jsonl(prompts, output_file):
    with open(output_file, 'w') as jsonl_file:
        for prompt in prompts:
            jsonl_file.write(json.dumps({"prompt": prompt}) + '\n')

    with open(output_file, 'r', encoding='utf-8') as jsonl_file:
        print(jsonl_file.read())


prompts = [
    "La frase {{text}} sostiene la frase {{hypothesis}}?",
    "La frase {{text}} implica la frase {{hypothesis}}?",
    "La frase {{hypothesis}} può essere dedotta dalla frase {{text}}?"
]

output_file = "textual_entailment-task1-prompt.jsonl"
generate_jsonl(prompts, output_file)
print(f"JSON Lines file '{output_file}' generated successfully with {len(prompts)} prompts.")


{"prompt": "La frase {{text}} sostiene la frase {{hypothesis}}?"}
{"prompt": "La frase {{text}} implica la frase {{hypothesis}}?"}
{"prompt": "La frase {{hypothesis}} pu\u00f2 essere dedotta dalla frase {{text}}?"}

JSON Lines file 'textual_entailment-task1-prompt.jsonl' generated successfully with 3 prompts.


# **3. Model  set up**

In [1]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or A100 to have sufficient RAM.

## 3.1 Install

In [5]:
!pip3 install -q -U transformers peft accelerate optimum
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/


Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

## 3.2 Load Model


- Evaluate diferent models

# **4. Evaluate Homework prompts**

## Test with  mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

## 4.2 Test the promps

In [2]:
import json
import torch

# Function to process each pair in the JSONL file
def visualize_prompt_response(jsonl_file):
    with open(jsonl_file, 'r') as file:
        count = 0  # Counter to track processed pairs
        for line in file:
            if count >= 5:  # Break the loop if 5 pairs have been processed
                break
            pair = json.loads(line)
            id = pair["id"]
            text = pair["text"]
            hypothesis = pair["hypothesis"]
            choices = pair["choices"]
            label = pair["label"]

            # Format the prompt with text and hypothesis
            prompt_formatted = f"La frase {text} sostiene la frase {hypothesis}?"
            
            output = classifier(prompt_formatted, choices, multi_label=False)
            sequence = output['sequence']
            selected_label = output['labels'][0] if output['scores'][0] > output['scores'][1] else output['labels'][1]
            accuracy = max(output['scores'])
            
            # Print the required information
            print("ID:", id)
            print("Sequence:", sequence)
            print("Selected Label:", selected_label)
            print("Accuracy:", round(accuracy, 3))
            print("Is Selected Label Correct:", selected_label == choices[label])
            print()

            count += 1  # Increment the counter for processed pairs

jsonl_file = 'textual_entailment-task1-train-data.jsonl'
visualize_prompt_response(jsonl_file)

ID: 0
Sequence: La frase 'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano. sostiene la frase 'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.?
Selected Label: implicato
Accuracy: 0.867
Is Selected Label Correct: True

ID: 1
Sequence: La frase Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni. sostiene la frase Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.?
Selected Label: implicato
Accuracy: 0.922
Is Selected Label Correct: True

ID: 2
Sequence: La frase Viene eletto sindaco di Catanzaro al ballottaggio del giugno del 2006, con il 50,8% dei voti. sostiene la frase Viene eletto sindaco di Catanzaro al ballottaggio del giugno 2006, con il 50,8% dei voti.?
Selected Label: implicato
Accuracy: 0.918
Is Selected Label Correct: True

ID: 3
Sequence: La frase Alle elezioni politiche del 9 aprile e 10 aprile 2006 sarà candidato alla Camera nella lista de L'Ulivo i

## 4.3. Compare the diferent prompts

In [3]:
# Prevent CUDA from running out of memory
import torch
torch.cuda.empty_cache()

- Test precision of one prmpt

In [4]:
# First test only with one prompt as example
import json

total_correct = 0
total_pairs = 0

# Load the JSONL file and process each pair
with open("textual_entailment-task1-test-data.jsonl", 'r') as file:
    for line in file:
        pair = json.loads(line)
        text = pair["text"]
        hypothesis = pair["hypothesis"]
        choices = pair["choices"]
        label = pair["label"]

        # Format the prompt with text and hypothesis
        prompt_formatted = f"La frase {text} sostiene la frase {hypothesis}?"
        
        output = classifier(prompt_formatted, choices, multi_label=False)
        sequence = output['sequence']
        selected_label = output['labels'][0] if output['scores'][0] > output['scores'][1] else output['labels'][1]
        accuracy = max(output['scores'])

        # Accumulate statistics
        if selected_label == choices[label]:
            total_correct += 1
        total_pairs += 1

    # Calculate overall statistics only if at least one pair was processed
    if total_pairs > 0:
        accuracy = total_correct / total_pairs * 100

        # Print statistics
        print("Overall Statistics:")
        print("Total Pairs:", total_pairs)
        print("Total Correct:", total_correct)
        print("Accuracy:", round(accuracy, 2), "%")
    else:
        print("No pairs found in the JSONL file.")

- Test precision of all the prompts

In [ ]:
# Test with all prompts
import json
import torch

def prompt_testing(prompt, file_path):
    total_correct = 0
    total_pairs = 0

    # Load the JSONL file and process each pair
    with open(file_path, 'r') as file:
        for line in file:
            pair = json.loads(line)
            text = pair["text"]
            hypothesis = pair["hypothesis"]
            choices = pair["choices"]
            label = pair["label"]

            # Format the prompt with text and hypothesis
            prompt_formatted = prompt.replace("{{text}}", text).replace("{{hypothesis}}", hypothesis)
            
            output = classifier(prompt_formatted, choices, multi_label=False)
            sequence = output['sequence']
            selected_label = output['labels'][0] if output['scores'][0] > output['scores'][1] else output['labels'][1]
            accuracy = max(output['scores'])

            # Accumulate statistics
            if selected_label == choices[label]:
                total_correct += 1
            total_pairs += 1

        # Calculate overall statistics only if at least one pair was processed
        if total_pairs > 0:
            accuracy = total_correct / total_pairs * 100

            # Print statistics
            print("Overall Statistics:")
            print("Total Pairs:", total_pairs)
            print("Total Correct:", total_correct)
            print("Accuracy:", round(accuracy, 2), "%")
        else:
            print("No pairs found in the JSONL file.")

# Load prompts from the JSONL file and test each prompt
with open("textual_entailment-task1-prompt.jsonl", 'r') as prompts_file:
    for line in prompts_file:
        prompt_data = json.loads(line)
        prompt = prompt_data["prompt"]
        print("\nPrompt:", prompt)
        file_path = "textual_entailment-task1-test-data.jsonl"
        prompt_testing(prompt, file_path)


Prompt: La frase {{text}} sostiene la frase {{hypothesis}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 219
Accuracy: 54.75 %

Prompt: La frase {{text}} implica la frase {{hypothesis}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 220
Accuracy: 55.0 %

Prompt: La frase {{hypothesis}} può essere dedotta dalla frase {{text}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 221
Accuracy: 55.25 %


## 4.4 Check the difference of accuracy between test and trainind dataset

In [ ]:
# Repeat test with training data instead of test data
with open("textual_entailment-task1-prompt.jsonl", 'r') as prompts_file:
    for line in prompts_file:
        prompt_data = json.loads(line)
        prompt = prompt_data["prompt"]
        print("\nPrompt:", prompt)
        input_file = "textual_entailment-task1-train-data.jsonl"
        prompt_testing(prompt, input_file)


Prompt: La frase {{text}} sostiene la frase {{hypothesis}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 200
Accuracy: 50.0 %

Prompt: La frase {{text}} implica la frase {{hypothesis}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 200
Accuracy: 50.0 %

Prompt: La frase {{hypothesis}} può essere dedotta dalla frase {{text}}?
Overall Statistics:
Total Pairs: 400
Total Correct: 200
Accuracy: 50.0 %
